I certify that the code and data in this assignment were generated independently, using only the tools
and resources defined in the course and that I did not receive any external help, coaching or contributions
during the production of this work.

  **PART 1**

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import gym
from gym import spaces
from matplotlib.offsetbox import (TextArea, DrawingArea, OffsetImage,AnnotationBbox)

In [1]:
class GridEnvironment(gym.Env):
    
    def __init__(self):
        self.observation_space = spaces.Discrete(16)
        self.action_space = spaces.Discrete(4)
        self.timestep = 0
        self.max_timesteps = 100
        
        self.max_episodes=1000
        self.max_epsi=1
        self.min_epsi=0.01
        self.decayfactor=0.01
        self.alpha=0.14
        self.gamma=0.6
        
    def reset(self):
        self.timestep = 0

        #count
        self.carrotcount=2
        self.hurdlecount=1
        self.trapcount=1

        #positions
        self.rabbit_pos = [0, 0]
        self.goal = [3, 3]
        self.trap= [1,1]
        self.hurdle=[3,2]
        self.carrot1=[1,3]
        self.carrot2=[2,0]
        self.carrot1_collect=False
        self.carrot2_collect=False
        self.hurdlegotrabbit=False
        self.trapgotrabbit=False
    
    def step(self, action,choice):
     
      done = False
      self.hurdlegotrabbit=False
      self.trapgotrabbit=False
      '''
        Defining the Stochastic environment

        Randomly selecting a probability between 0.1 and 0.9
        if it is greater than or equal to 0.8 choosing a new action leaving the choosen action
      '''

      if choice == 0:
        prob=random.randint(1,9)/10
        if(prob>=0.8):
          start=0
          end=4
          a=list(range(start,action))+list(range(action+1,end))
          action=random.choice(a)

      """
        action=0 up
        action=1 down
        action=2 left
        action=3 right
      """  
  
      if action == 0:
        self.rabbit_pos[1] += 1
      if action == 1:
        self.rabbit_pos[1] -= 1
      if action == 2:
        self.rabbit_pos[0] -= 1
      if action == 3:
        self.rabbit_pos[0] += 1

      #Clipping the position so that the agent doesn't go out of the grid
      self.rabbit_pos = np.clip(self.rabbit_pos, 0, 3)
      
      reward = 0
      if (self.rabbit_pos == self.goal).all():
        reward = 2
        done=True
      elif (self.rabbit_pos == self.trap).all() and self.trapgotrabbit==False and self.trapcount>0:
        reward = -1
        self.trapgotrabbit=True
        self.trapcount -=1
      elif (self.rabbit_pos == self.hurdle).all() and self.hurdlegotrabbit==False and self.hurdlecount>0:
        reward = -0.5
        self.hurdlegotrabbit=True
        self.hurdlecount -=1
      elif (self.rabbit_pos == self.carrot1).all() and self.carrot1_collect==False and self.carrotcount>0:
        self.carrot1_collect=True
        reward = 1
        self.carrotcount -=1
      elif (self.rabbit_pos == self.carrot2).all() and self.carrot2_collect==False and self.carrotcount>0:
        self.carrot2_collect=True
        reward = 1
        self.carrotcount -=1
         
      self.timestep += 1
      done = True if self.timestep >= self.max_timesteps or done==True else False
      return reward, done
        
    def render(self):
      fig,ax=plt.subplots(figsize=(10,10))
      ax.set_xlim(0,4)
      ax.set_ylim(0,4)

      def plot_image(plot_pos):
        
        plot_rabbit,plot_hurdle,plot_trap,plot_carrot1,plot_carrot2,plot_goal=False,False,False,False,False,False

        if (self.rabbit_pos == plot_pos).all():
          plot_rabbit=True
        if (self.hurdle == plot_pos).all() :
          plot_hurdle=True
        if (self.trap== plot_pos).all() :
          plot_trap=True
        if (self.carrot1 == plot_pos).all() and self.carrot1_collect==False:
          plot_carrot1=True
        if (self.carrot2 == plot_pos).all() and self.carrot2_collect==False:
          plot_carrot2=True
        if (self.goal == plot_pos).all():
          plot_goal=True

        #plot for rabbit
        if plot_rabbit and all(not item for item in [plot_hurdle,plot_goal,plot_carrot1,plot_carrot2,plot_trap]):
          rabbit=AnnotationBbox(OffsetImage(plt.imread('./images/rabbit.jpg'),zoom=0.28),np.add(plot_pos,[0.5,0.5]),frameon=False)
          ax.add_artist(rabbit)

        #plot for trap
        elif plot_trap and self.trapcount>0 and all(not item for item in [plot_rabbit,plot_hurdle,plot_goal,plot_carrot1,plot_carrot2]):
          trap=AnnotationBbox(OffsetImage(plt.imread('./images/trap.jpg'),zoom=0.2),np.add(plot_pos,[0.5,0.5]),frameon=False)
          ax.add_artist(trap)
        
        #plot for hurdle
        elif plot_hurdle and self.hurdlecount>0 and all(not item for item in [plot_rabbit,plot_trap,plot_goal,plot_carrot1,plot_carrot2]):
          hurdle=AnnotationBbox(OffsetImage(plt.imread('./images/hurdle.jpg'),zoom=0.37),np.add(plot_pos,[0.5,0.5]),frameon=False)
          ax.add_artist(hurdle)
        
        #plot for carrot1
        elif plot_carrot1  and all(not item for item in [plot_rabbit,plot_hurdle,plot_goal,plot_carrot2,plot_trap]):
          carrot=AnnotationBbox(OffsetImage(plt.imread('./images/carrot.png'),zoom=0.12),np.add(plot_pos,[0.5,0.5]),frameon=False)
          ax.add_artist(carrot)
        
        #plot for carrot2
        elif plot_carrot2 and all(not item for item in [plot_rabbit,plot_hurdle,plot_goal,plot_carrot1,plot_trap]):
          carrot=AnnotationBbox(OffsetImage(plt.imread('./images/carrot.png'),zoom=0.12),np.add(plot_pos,[0.5,0.5]),frameon=False)
          ax.add_artist(carrot)
        
        #plot for goal
        elif plot_goal and all(not item for item in [plot_rabbit,plot_hurdle,plot_carrot2,plot_trap,plot_carrot1]):
          goal=AnnotationBbox(OffsetImage(plt.imread('./images/goal.jpg'),zoom=0.25),np.add(plot_pos,[0.5,0.5]),frameon=False)
          ax.add_artist(goal)

        #plot for rabbit getting trapped
        elif all(item for item in[plot_rabbit,plot_trap]) and self.trapgotrabbit==True and all(not item for item in [plot_hurdle,plot_goal,plot_carrot1,plot_carrot2]):
          rabbit_trap=AnnotationBbox(OffsetImage(plt.imread('./images/rabbit_trap.png'),zoom=0.7),np.add(plot_pos,[0.5,0.5]),frameon=False)
          ax.add_artist(rabbit_trap)
        elif all(item for item in[plot_rabbit,plot_trap]) and self.trapgotrabbit==False and all(not item for item in [plot_hurdle,plot_goal,plot_carrot1,plot_carrot2]):
          rabbit=AnnotationBbox(OffsetImage(plt.imread('./images/rabbit.jpg'),zoom=0.28),np.add(plot_pos,[0.5,0.5]),frameon=False)
          ax.add_artist(rabbit)

        #plot for rabbit got to a hurdle
        elif all(item for item in[plot_rabbit,plot_hurdle]) and self.hurdlegotrabbit==True and all(not item for item in [plot_trap,plot_goal,plot_carrot1,plot_carrot2]):
          rabbit_hurdle=AnnotationBbox(OffsetImage(plt.imread('./images/rabbit_hurdle.jpeg'),zoom=0.1),np.add(plot_pos,[0.5,0.5]),frameon=False)
          ax.add_artist(rabbit_hurdle)
        elif all(item for item in[plot_rabbit,plot_hurdle]) and self.hurdlegotrabbit==False and all(not item for item in [plot_trap,plot_goal,plot_carrot1,plot_carrot2]):
          rabbit=AnnotationBbox(OffsetImage(plt.imread('./images/rabbit.jpg'),zoom=0.28),np.add(plot_pos,[0.5,0.5]),frameon=False)
          ax.add_artist(rabbit)

        #rabbit reaching the goal
        elif all(item for item in[plot_rabbit,plot_goal]) and all(not item for item in [plot_trap,plot_hurdle,plot_carrot1,plot_carrot2]):
          rabbit_goal=AnnotationBbox(OffsetImage(plt.imread('./images/rabbit_goal.png'),zoom=0.4),np.add(plot_pos,[0.5,0.5]),frameon=False)
          ax.add_artist(rabbit_goal)
    
      state_mapping={}
      for j in range(16):
        state_mapping[j]=np.asarray([j%4,int(np.floor(j/4))])

      for position in state_mapping:
        plot_image(state_mapping[position])
    
      plt.xticks([0,1,2,3,4])
      plt.yticks([0,1,2,3,4])
      plt.grid()
      plt.show()

NameError: name 'gym' is not defined

In [ ]:
class RandomAgent:
  def __init__(self, env):
    self.env = env
    self.observation_space = env.observation_space
    self.action_space = env.action_space

  def step(self, observation):
    return np.random.choice(self.action_space.n)

In [ ]:
env = GridEnvironment()
agent = RandomAgent(env)

env.reset()
env.max_timesteps = 10
done = False
while not done:
  action = agent.step(env)
  """
  choice = 0 Stochastic
  choice = 1 Deterministic
  """
  reward, done = env.step(action,choice=1)
  env.render()

**e-greedy algorithm**

In [ ]:
def e_greedy(Q, state,nactions , epsi):
    #Exploration
    if random.uniform(0, 1)  < epsi:
        action = np.random.randint(0, nactions)

    #Exploitation
    else:
        val = max(Q[tuple(state)].values())
        action = np.random.choice([i for i, j in Q[tuple(state)].items() if j == val])

    return action

**Q Learning**


In [ ]:
def qlearning(env,choice):
    rewards_episode = []  
    epsilons = []
    Q = dict()
    for row in range(4):
        for column in range(4):
                Q[(row,column)] = {0:0, 1:0, 2:0, 3:0}
    epsilon=1
    for i in range(env.max_episodes):
        cummu_reward=0
        env.reset()
        done=False
        count=0
        while not done:
            old_state = env.rabbit_pos.copy()
            action=e_greedy(Q,old_state,env.action_space.n,epsilon)
            #print(action)
            #print(env.rabbit_pos)
            reward, done= env.step(action)
            #print(Q[tuple(env.rabbit_pos)].values())
            new_state=env.rabbit_pos
            Q[tuple(old_state)][action] += env.alpha * (reward + env.gamma * max(Q[tuple(new_state)].values()) - Q[tuple(old_state)][action])
            cummu_reward+=reward
            old_state=env.rabbit_pos.copy()
            count+=1
        rewards_episode.append(cummu_reward)
        epsilon = env.min_epsi + (env.max_epsi - env.min_epsi)*np.exp(-env.decayfactor*i)
        epsilons.append(epsilon)
        #print(count)
    return Q,rewards_episode,epsilons

**Deterministic Qlearning**

In [ ]:
#Deterministic Q-Learning
env = GridEnvironment()
env.reset()
optimalQ,rewardsepisode,epsilons=qlearning(env,choice=1)

#Plotting

fig = plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
x = range(len(rewardsepisode))
plt.plot(x,rewardsepisode)
plt.title('Total Reward per Episode')
plt.ylabel('Rewards')
plt.xlabel('Episodes')


plt.subplot(1,2,2)
x1= range(len(epsilons))
plt.plot(x1,epsilons)
plt.title('Epsilon decay')
plt.ylabel('Epsilons')
plt.xlabel('Episodes')

fig.tight_layout()
plt.show()

In [ ]:
env=GridEnvironment()
epsireward=[]
for i in range(1,11):
  env.reset()
  done = False
  r=0
  env.max_timesteps=10
  print('EPISODE:',i)
  while not done:
    val = max(optimalQ[tuple(env.rabbit_pos)].values())
    actions=optimalQ[tuple(env.rabbit_pos)]
    action=max(actions,key=actions.get)
    """
    choice = 0 Stochastic
    choice = 1 Deterministic
    """
    reward, done = env.step(action,choice=1)
    r+=reward
    env.render()
  epsireward.append(r)

In [ ]:
x = range(len(epsireward))
plt.plot(x,epsireward)
plt.title('Total Reward per Episode')
plt.ylabel('Rewards')
plt.xlabel('Episodes')

**Stochastic Qlearning**

In [ ]:
#Stochastic Q-Learning
env = GridEnvironment()
env.reset()
optimalQ,rewardsepisode,epsilons=qlearning(env,choice=0)

#Plotting

fig = plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
x = range(len(rewardsepisode))
plt.plot(x,rewardsepisode)
plt.title('Total Reward per Episode')
plt.ylabel('Rewards')
plt.xlabel('Episodes')


plt.subplot(1,2,2)
x1= range(len(epsilons))
plt.plot(x1,epsilons)
plt.title('Epsilon decay')
plt.ylabel('Epsilons')
plt.xlabel('Episodes')

fig.tight_layout()
plt.show()

In [ ]:
env=GridEnvironment()
sqrewards=[]
doneq=[]
for i in range(1,11):
  print('EPISODE:',i)
  env.reset()
  done = False
  env.max_timesteps=12
  rs=0
  while not done:
    val = max(optimalQ[tuple(env.rabbit_pos)].values())
    actions=optimalQ[tuple(env.rabbit_pos)]
    action=max(actions,key=actions.get)
    #action = np.random.choice([i for i, j in optimalQ[tuple(env.rabbit_pos)].items() if j == val])
    """
    choice = 0 Stochastic
    choice = 1 Deterministic
    """
    reward, done = env.step(action,choice=0)
    rs+=reward
    env.render()
  sqrewards.append(rs)
  doneq.append(done)

In [ ]:
x = range(len(sqrewards))
plt.plot(x,sqrewards)
plt.title('Total Reward per Episode')
plt.ylabel('Rewards')
plt.xlabel('Episodes')

**SARSA**

In [ ]:
def sarsa(env,choice):
    rewards_episode = []  
    epsilons = []
    Qs = dict()
    for row in range(4):
        for column in range(4):
                Qs[(row,column)] = {0:0, 1:0, 2:0, 3:0}
    epsilon=1
    for i in range(env.max_episodes):
        cummu_reward=0
        env.reset()
        done=False
        count=0
        old_state = env.rabbit_pos.copy()
        action=e_greedy(Qs,old_state,env.action_space.n,epsilon)
        while not done:
            reward, done= env.step(action,choice)
            new_state=env.rabbit_pos.copy()
            new_action=e_greedy(Qs,new_state,env.action_space.n,epsilon)
            Qs[tuple(old_state)][action] += env.alpha * (reward + env.gamma * Qs[tuple(new_state)][new_action] - Qs[tuple(old_state)][action])
            cummu_reward+=reward
            old_state=env.rabbit_pos.copy()
            action=new_action
            count+=1
        rewards_episode.append(cummu_reward)
        epsilon = env.min_epsi + (env.max_epsi - env.min_epsi)*np.exp(-env.decayfactor*i)
        epsilons.append(epsilon)
        #print(count)
    #print(rewards_episode)
    return Qs,rewards_episode,epsilons

**Deterministic SARSA**

In [ ]:
#Deterministic SARSA
env = GridEnvironment()
env.reset()
optimalQs,sdrewards,sdepsilons=sarsa(env,choice=1)

#Plotting

fig = plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
x = range(len(sdrewards))
plt.plot(x,sdrewards)
plt.title('Total Reward per Episode')
plt.ylabel('Rewards')
plt.xlabel('Episodes')


plt.subplot(1,2,2)
x1= range(len(sdepsilons))
plt.plot(x1,sdepsilons)
plt.title('Epsilon decay')
plt.ylabel('Epsilons')
plt.xlabel('Episodes')

fig.tight_layout()
plt.show()

In [ ]:
env=GridEnvironment()
sdrew=[]
for i in range(1,11):
  print('EPISODE:',i)
  env.reset()
  done = False
  env.max_timesteps=10
  sdr=0
  while not done:
    val = max(optimalQs[tuple(env.rabbit_pos)].values())
    actions=optimalQs[tuple(env.rabbit_pos)]
    action=max(actions,key=actions.get)
    """
    choice = 0 Stochastic
    choice = 1 Deterministic
    """
    reward, done = env.step(action,choice=1)
    sdr+=reward
    env.render()
  sdrew.append(sdr)

In [ ]:
x = range(len(sdrew))
plt.plot(x,sdrew)
plt.title('Total Reward per Episode')
plt.ylabel('Rewards')
plt.xlabel('Episodes')

**Stochastic SARSA**

In [ ]:
#Stochastic SARSA
env = GridEnvironment()
env.reset()
optimalQss,ssrewards,ssepsilons=sarsa(env,choice=0)

#Plotting

fig = plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
x = range(len(ssrewards))
plt.plot(x,ssrewards)
plt.title('Total Reward per Episode')
plt.ylabel('Rewards')
plt.xlabel('Episodes')


plt.subplot(1,2,2)
x1= range(len(ssepsilons))
plt.plot(x1,ssepsilons)
plt.title('Epsilon decay')
plt.ylabel('Epsilons')
plt.xlabel('Episodes')

fig.tight_layout()
plt.show()

In [ ]:
env=GridEnvironment()
ssrew=[]
dones=[]
for i in range(1,11):
  print('EPISODE:',i)
  env.reset()
  done = False
  env.max_timesteps=12
  ssr=0
  while not done:
    val = max(optimalQss[tuple(env.rabbit_pos)].values())
    actions=optimalQss[tuple(env.rabbit_pos)]
    action=max(actions,key=actions.get)
    """
    choice = 0 Stochastic
    choice = 1 Deterministic
    """
    reward, done = env.step(action,choice=0)
    ssr+=reward
    env.render()
  ssrew.append(ssr)
  dones.append(done)

In [ ]:
x = range(len(ssrew))
plt.plot(x,ssrew)
plt.title('Total Reward per Episode')
plt.ylabel('Rewards')
plt.xlabel('Episodes')

**Qlearning VS SARSA**

In [ ]:
fig = plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(x,epsireward)
plt.plot(x,sdrew)
plt.title('Qlearning vs SARSA (Deterministic)')
plt.ylabel('Rewards')
plt.xlabel('Episodes')
plt.legend(['Qlearning', 'SARSA'], loc='upper left')


plt.subplot(1,2,2)
plt.plot(x,sqrewards)
plt.plot(x,ssrew)
plt.title('Qlearning vs SARSA (Stochastic')
plt.ylabel('Rewards')
plt.xlabel('Episodes')
plt.legend(['Qlearning', 'SARSA'], loc='upper left')

fig.tight_layout()
plt.show()